In [61]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [62]:
# Douwload the data set of iris
iris = load_iris()
X = iris.data
y = iris.target
feature_names = iris.feature_names
class_names = iris.target_names

print('Features:', feature_names)
print('Classes:', class_names)

Features: ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
Classes: ['setosa' 'versicolor' 'virginica']


In [63]:
# Split the data into training and test data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print('\nThe size of train set:', X_train.shape)
print('The size of test set:', X_test.shape)


The size of train set: (105, 4)
The size of test set: (45, 4)


In [64]:
# Feature selection separately for each class
classes = np.unique(y_train)
n_classes = len (classes)
n_features = X_train.shape[1]

#Split the data on classes
X_by_class = [X_train[y_train == c] for c in classes]

In [65]:
# Calculate covariance matrices for each class
cov_matrices = [np.cov(X_by_class[c].T) for c in range(n_classes)]
print('\nMatrices covariance for each clas:')
for i, cov in enumerate(cov_matrices):
    print(f'\nClass {class_names[i]}:')
    print(cov)


Matrices covariance for each clas:

Class setosa:
[[0.11569892 0.09817204 0.01669892 0.00677419]
 [0.09817204 0.14113978 0.01950538 0.00712903]
 [0.01669892 0.01950538 0.03436559 0.00877419]
 [0.00677419 0.00712903 0.00877419 0.01191398]]

Class versicolor:
[[0.28297297 0.08816817 0.19847598 0.05760511]
 [0.08816817 0.08966967 0.09222973 0.04215465]
 [0.19847598 0.09222973 0.24599099 0.08385886]
 [0.05760511 0.04215465 0.08385886 0.04249249]]

Class virginica:
[[0.43414414 0.09777027 0.31913664 0.04939189]
 [0.09777027 0.09897898 0.08758258 0.06146396]
 [0.31913664 0.08758258 0.29644144 0.05224474]
 [0.04939189 0.06146396 0.05224474 0.0883033 ]]


In [66]:
# Calculation inverse covariance matrices
inv_cov_matrices = [np.linalg.inv(cov) for cov in cov_matrices]

In [67]:
#Calculation the prior probabilities of each class
priors = [np.mean(y_train == c) for c in classes]

In [68]:
#The function to calculate discriminant function for a single row
def quadratic_discriminant_function(x, mean, inv_cov, prior, class_name):
  diff = x - mean
  # quadratic form:-1/2 * (x-μ)^T * Σ^-1 * (x-μ)
  quadratic = -0.5 * np.dot(np.dot(diff.T, inv_cov), diff)
  # The logarithm of the prior probabilities and determinant of the covariant matrice
  log_term = np.log(prior) - 0.5 * np.log(np.linalg.det(np.linalg.inv(inv_cov)))
  return quadratic + log_term

In [69]:
# The function to calculate discriminant functions and probabilities for entire matrix
def predict_proba_custom(X_test, means, inv_covs, priors):
    n_samples = X_test.shape[0]
    n_classes = len(priors)

  # Calculate values of discriminant function for each class
    discriminant_values = np.zeros((n_samples, n_classes))

    for i in range(n_samples):
          for c in range(n_classes):
              discriminant_values[i, c] = quadratic_discriminant_function(X_test[i], means[c], inv_covs[c], priors[c], class_names[c])

  # Convert to probabilities using softmax
    max_values = np.max(discriminant_values, axis=1, keepdims=True)
    exp_values = np.exp(discriminant_values - max_values) # for numerical stability
    proba = exp_values / np.sum(exp_values, axis = 1, keepdims = True)
    return proba

# calculate average values for each class
means = [np.mean(X_by_class[c], axis=0) for c in range(n_classes)]

# predict of probabilities using our function
proba_custom = predict_proba_custom(X_test, means, inv_cov_matrices, priors)
predictions_custom = np.argmax(proba_custom, axis=1)

In [70]:
# Prediction using QuadraticDiscriminantAnalisys using sklearn
qda = QuadraticDiscriminantAnalysis()
qda.fit(X_train, y_train)
proba_sklearn = qda.predict_proba(X_test)
predictions_sklearn = qda.predict(X_test)


In [71]:
# Comparison of results
print("\nComparison of results")
print('The accuracy of our implementation:', accuracy_score(y_test, predictions_custom))
print("The accuracy sklearn QDA:", accuracy_score(y_test, predictions_sklearn))

print("\nExample prediction probabilities:")
for i in range(5): #Output the first examples
    print(f"\nThe example {i+1}:")
    print("Our implementation:", proba_custom[i])
    print('Sklearn: ', proba_sklearn[i])

# Conclusion on the similarity of results
print("\nConclusion:")
print("Results of ours implementation QDA and implementation with sklearn very similar.")
print("The clasification accuracy is almost identical, confirming the correctness of our implementation")
print("Small differences in probabilities may be due to implementation specifics (e.g., handling of numerical stability).")


Comparison of results
The accuracy of our implementation: 1.0
The accuracy sklearn QDA: 1.0

Example prediction probabilities:

The example 1:
Our implementation: [1.09461742e-81 9.72753474e-01 2.72465259e-02]
Sklearn:  [1.09461742e-81 9.72753474e-01 2.72465259e-02]

The example 2:
Our implementation: [1.00000000e+00 5.73230921e-28 3.05634855e-63]
Sklearn:  [1.00000000e+00 5.73230921e-28 3.05634855e-63]

The example 3:
Our implementation: [2.43687730e-251 4.97846558e-009 9.99999995e-001]
Sklearn:  [2.43687730e-251 4.97846558e-009 9.99999995e-001]

The example 4:
Our implementation: [5.10831398e-77 9.97298384e-01 2.70161647e-03]
Sklearn:  [5.10831398e-77 9.97298384e-01 2.70161647e-03]

The example 5:
Our implementation: [1.26234369e-98 9.99396589e-01 6.03410624e-04]
Sklearn:  [1.26234369e-98 9.99396589e-01 6.03410624e-04]

Conclusion:
Results of ours implementation QDA and implementation with sklearn very similar.
The clasification accuracy is almost identical, confirming the correctne